In [14]:
%load_ext autoreload
%autoreload 2

from sqlalchemy.orm import sessionmaker
from hansard.models import MP, Debate, SpokenContribution, Party, db_connect, create_table
import hansard.items

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
mp = hansard.items.MP(
        name = "Dave",
        start_year = "1990",
        end_year = "present",
        constituency_last = "Derkenshire",
        house = "Upper",
        party = "Epic"
        )

In [8]:
class HansardPipeline(object):
    def __init__(self):
        engine = db_connect()
        create_table(engine)
        self.Session = sessionmaker(bind=engine)

    def process_item(self, item):
        session = self.Session()
        print("Attempting to add to DB")

        try:
            if type(item) is hansard.items.MP:
                mp = MP(**item)
                try:
                    session.add(mp)
                    session.commit()
                    print("Sucess!!")
                except:
                    session.rollback()
                    print("Failure...")
                finally:
                    session.close()
                    print("All done")
        except:
            raise
        return item

In [16]:
pipeline = HansardPipeline()

In [28]:
pipeline.process_item(mp)

Attempting to add to DB


InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers.  Original exception was: Could not determine join condition between parent/child tables on relationship Party.mps - there are no foreign keys linking these tables.  Ensure that referencing columns are associated with a ForeignKey or ForeignKeyConstraint, or specify a 'primaryjoin' expression.